In [21]:
import numpy as np
import os, re

In [22]:
SAMPLES = 32

In [23]:
def process_cir_data(data):
    if len(data)!=SAMPLES*6*2:
        raise BaseException('CIR data incomplete...')
    data_array = []
    for i in range(0, len(data), 6):
        # [low mid high]
        real_part = [data[i], data[i+1], data[i+2]]
        imag_part = [data[i+3], data[i+4], data[i+5]]
        real_num = int24to32(real_part)
        imag_num = int24to32(imag_part)
        data_array.append(real_num+1j*imag_num)
    return data_array

def int24to32(arr):
    num = np.int32(0)
    # print(f"{arr[2]: b}")
    if not arr[2]&0b1000_0000:
        # positive
        num = ((((num|arr[2])<<8)|arr[1])<<8)|arr[0]
    else:
        # negative
        # 11111111 00000000 00000000 00000000
        # 00000000 11111111 11111111 11111111
        # 00000001 00000000 00000000 00000000
        # num_mask = np.int32(-2**24)
        num = (((((num|arr[2])<<8)|arr[1])<<8)|arr[0])<<8
        # num = num|num_mask
        num = (~(num-1))>>8
        num = -num
        # -8388608
    return num

import numpy as np

def fixed_point_to_decimal(fixed_point_data):
    integer_part = fixed_point_data >> 6
    decimal_part = (fixed_point_data & 0x3F) / (1 << 6)
    return integer_part + decimal_part


In [24]:
# test_str = rb'''TAG_ID:0000;SEQ:52;CIR_PDOA:-10.826888,sts_Power:6,sts_F1:6639,sts_F2:6549,sts_F3:2199,sts_FpIndex:22658,sts_AccumCount:32,sts_Peak_Index:198,sts_Peak_Amp:6639,sts2_Power:9,sts2_F1:6978,sts2_F2:6497,sts2_F3:3560,sts2_FpIndex:22661,sts2_AccumCount:32,sts2_Peak_Index:198,sts2_Peak_Amp:6639,sts_quality:3,sts_quality_index:60,
# P_FIN:-16.254320;P_POLL:-11.106652;AOA:-4.588250;DIST:440.000000;
# CIR:123END'''

# p = re.compile(b'sts_FpIndex:(.+?),.+?sts2_FpIndex:(.+?),.+?CIR:(.+?)END', flags=re.DOTALL)

# match = re.findall(p, test_str)

# print(match)

In [25]:
root = "raw/"
files = os.listdir(root)

print(files)

data = []
for file in files:
    with open(root + file, 'rb') as f:
        content = f.read()
        
        p = re.compile(b'sts_FpIndex:(.+?),.+?sts2_FpIndex:(.+?),.+?CIR:(.+?)END', flags=re.DOTALL)
        match = re.findall(p, content)
        processed_match = []
        for i in range(len(match)):
            sts1_fp_index = int(match[i][0])
            sts2_fp_index = int(match[i][1])
            cir = match[i][2]
            processed_match.append([sts1_fp_index, sts2_fp_index, cir])
        data.append(processed_match)
    # break

['CIR_data_LOS_1_1.txt', 'CIR_data_LOS_1_2.txt', 'CIR_data_LOS_2_1.txt', 'CIR_data_LOS_2_2.txt', 'CIR_data_LOS_3_1.txt', 'CIR_data_LOS_3_2.txt', 'CIR_data_LOS_4_1.txt', 'CIR_data_LOS_4_2.txt', 'CIR_data_NLOS_1_1.txt', 'CIR_data_NLOS_1_2.txt', 'CIR_data_NLOS_2_1.txt', 'CIR_data_NLOS_2_2.txt', 'CIR_data_NLOS_3_1.txt', 'CIR_data_NLOS_3_2.txt', 'CIR_data_NLOS_4_1.txt', 'CIR_data_NLOS_4_2.txt']


In [26]:
print(data[0][1])
print(len(data[0][1]))

[22658, 22661, b'\xad\xf2\xff\xff\x0b\x00\x00\xea\xff\xff\x1b\x00\x00\xf3\xff\xff\x12\x00\x00\xeb\xff\xff*\x00\x00\x03\x00\x00\x03\x00\x00\xf6\xff\xff\xdd\xff\xff\xfe\xff\xff\xfa\xff\xff\xde\xff\xff\xf3\xff\xff\xfc\xff\xff\x16\x00\x00\x15\x00\x00\xf5\xff\xff\t\x00\x00\x02\x00\x00\x04\x00\x00\x19\x00\x00\r\x00\x00\x15\x00\x00\x0f\x00\x00\xea\xff\xff\x06\x00\x00\x00\x00\x00\x05\x00\x00\x03\x00\x00\xef\xff\xff\x07\x00\x00\xf7\xff\xff\x06\x00\x00\xfe\xff\xff\x05\x00\x00\x01\x00\x00\x18\x00\x00\x04\x00\x00\xf4\xff\xff\xdd\xff\xff\xf6\xff\xff\xdb\xff\xff\xe8\xff\xff\x03\x00\x00\x12\x00\x00\xf0\xff\xff\xf6\xff\xff\x0c\x00\x00\x12\x00\x00\x02\x00\x00\xf8\xff\xff\x10\x00\x00\x1d\x00\x00\xf4\xff\xff\xfa\xff\xff\xfe\xff\xff\x12\x00\x00\x05\x00\x00\n\x00\x00\x0f\x00\x00\xfd\xff\xff1\x00\x00\xef\xff\xff.\x00\x00\xef\xff\xff*\x00\x00\xe1\xff\xff\x15\x00\x00\x18\x00\x00\r\x00\x00\x15\x00\x00\x0c\x00\x007\x00\x00\xff\xff\xff\x11\x00\x00\xe7\xff\xff\x07\x00\x00\xf4\xff\xff\xe5\xff\xff\xf3\xff\xff\xe1\x

In [27]:
# 示例用法
fixed_point_data = np.uint16(22658)  # 用作示例的固定点数据
result = int(fixed_point_to_decimal(fixed_point_data))
print("转换结果:", result)

# 示例用法
fixed_point_data = np.uint16(22661)  # 用作示例的固定点数据
result = int(fixed_point_to_decimal(fixed_point_data))
print("转换结果:", result)

转换结果: 354
转换结果: 354


In [28]:
def get_valid_cir(cir_bytes, sts1_fp_data, sts2_fp_data):
    # 1024 samples, 6 = (3 bytes * 2) per sample, 1 dummy byte
    samples = 1024
    bytes_per_sample = 3 * 2    
    if (len(cir_bytes)!=samples * bytes_per_sample + 1):
        raise BaseException('CIR data incomplete...')

    # 从那个结构体里面获取，对应sts1_FpIndex，sts2_FpIndex

    sts1_fp = int(fixed_point_to_decimal(sts1_fp_data))
    sts2_fp = int(fixed_point_to_decimal(sts2_fp_data))

    # sts sts2各取128个samples
    span_per_sts = SAMPLES
    # 从fp-32到fp-32+128
    offset_to_fp = -8

    sts1_fp_index = 1 + bytes_per_sample * (sts1_fp + offset_to_fp)
    sts2_fp_index = 1 + bytes_per_sample * (sts2_fp + offset_to_fp)

    sts1 = cir_bytes[sts1_fp_index:sts1_fp_index+span_per_sts * bytes_per_sample]
    sts2 = cir_bytes[sts2_fp_index:sts2_fp_index+span_per_sts * bytes_per_sample]
    
    sts = sts1 + sts2
    return  sts


In [29]:
x = np.empty((0, SAMPLES*2))
y = np.empty((0, ))
x_set = []
y_set = []
all_cir = []
for k in range(len(data)):
    dataset = data[k]
    n = len(dataset)
    processed_data = []
    for i in range(n):
        sts1_fp_index = dataset[i][0]
        sts2_fp_index = dataset[i][1]
        raw_cir = dataset[i][2]
        # print(sts1_fp_index, sts2_fp_index, len(raw_cir))
        try:
            valid_raw_cir = get_valid_cir(raw_cir, sts1_fp_index, sts2_fp_index)
            cir = process_cir_data(valid_raw_cir)
            processed_data.append(cir)
        except:
            pass
            # print(i)
    print(len(processed_data))
    all_cir.append(processed_data)
    
    _x = np.array(processed_data)
    _y = np.zeros((_x.shape[0],), dtype=int) if "NLOS" in files[k] else np.ones((_x.shape[0],), dtype=int)
    print(_x.shape, _y.shape)
    x = np.concatenate((x, _x), axis = 0)
    y = np.concatenate((y, _y), axis = 0)
    x_set.append(_x)
    y_set.append(_y)
    
print(len(all_cir[0]))

3588
(3588, 64) (3588,)
3659
(3659, 64) (3659,)
3503
(3503, 64) (3503,)
3527
(3527, 64) (3527,)
4261
(4261, 64) (4261,)
3517
(3517, 64) (3517,)
3507
(3507, 64) (3507,)
5685
(5685, 64) (5685,)
3706
(3706, 64) (3706,)
4505
(4505, 64) (4505,)
3845
(3845, 64) (3845,)
3756
(3756, 64) (3756,)
3737
(3737, 64) (3737,)
3615
(3615, 64) (3615,)
3629
(3629, 64) (3629,)
3674
(3674, 64) (3674,)
3588


In [30]:
print(x.shape)
print(y.shape)
cir = x[0]
label = y[0]
print(cir)
print(label)
cir_amp = np.abs(cir)
# fp1 和 fp2太近了，为什么
# import matplotlib.pyplot as plt
# plt.figure()
# plt.plot(np.arange(len(cir_amp)), cir_amp)
# plt.show()

(61714, 64)
(61714,)
[  11.  +31.j  -24.  +33.j  -31.  +37.j    6.  +33.j    9.   +7.j
   44.   +3.j  168.  +92.j  805. +369.j 1544. +463.j 1581. +225.j
  514. +143.j -373. +636.j -708.+1113.j -299. +783.j   86. +119.j
  292. -174.j  473. -421.j  662. -595.j  508. -528.j   15. -205.j
 -446.  +19.j -517.  +97.j -163.  +29.j  212.   -9.j  264.  +54.j
  146. +148.j  -43. +214.j -124. +565.j   57. +747.j  101. +669.j
   -5. +242.j  -90.  +24.j  -24.  +33.j  -31.  +37.j    6.  +33.j
    9.   +7.j   44.   +3.j  168.  +92.j  805. +369.j 1544. +463.j
 1581. +225.j  514. +143.j -373. +636.j -708.+1113.j -299. +783.j
   86. +119.j  292. -174.j  473. -421.j  662. -595.j  508. -528.j
   15. -205.j -446.  +19.j -517.  +97.j -163.  +29.j  212.   -9.j
  264.  +54.j  146. +148.j  -43. +214.j -124. +565.j   57. +747.j
  101. +669.j   -5. +242.j  -90.  +24.j  195. -346.j]
1.0


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from helper import gen_I_Q_Amp_mat, gen_I_Q_Amp_Angle_mat

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using:", device)

using: cuda


In [32]:
def concatenate_data(x, y):
    merged_x = np.empty((0, SAMPLES*2))
    merged_y = np.empty((0, ))
    
    for i in range(len(x)):
        merged_x = np.concatenate((merged_x, x[i]), axis = 0)
        merged_y = np.concatenate((merged_y, y[i]), axis = 0)
        
    return merged_x, merged_y

def prepare_train_dataset(x, y, l, dim = 1):
    val_idx = [2, 13]
    
    # 检查是否有可用的GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("using:", device)
    
    print(f"validate with {[l[i] for i in val_idx]}")

    # print(len(x), len(y))

    # 3 CIR_data_NLOS_2_2.npz
    # 5 CIR_data_LOS_3_2.npz
    
    # for idx, name in enumerate(l):
    #     print(idx, name)
        
    def normalize_x(x):
        x_amp = np.abs(x)
        x_span = np.max(x_amp, axis=1) - np.min(x_amp, axis=1)

        x_norm = x.T/x_span.T
        x_norm = x_norm.T
        return x_norm

    x_norm = []
    for i in x:
        x_norm.append(normalize_x(i))
        
    x = x_norm

    x_prior = [data for idx, data in enumerate(x) if idx not in val_idx]
    y_prior = [data for idx, data in enumerate(y) if idx not in val_idx]
    x_post = [data for idx, data in enumerate(x) if idx in val_idx]
    y_post = [data for idx, data in enumerate(y) if idx in val_idx]
    x_train, y_train = concatenate_data(x_prior, y_prior)
    x_val, y_val = concatenate_data(x_post, y_post)

    total_len = len(y)
    
    if dim==1:
        x_train = np.abs(x_train)
        x_val = np.abs(x_val)
    elif dim==3:
        x_train = gen_I_Q_Amp_mat(x_train)
        x_val = gen_I_Q_Amp_mat(x_val)
    elif dim==4:
        x_train = gen_I_Q_Amp_Angle_mat(x_train)
        x_val = gen_I_Q_Amp_Angle_mat(x_val)

    # Convert numpy arrays to PyTorch tensors (use torch.LongTensor for indices)
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32, device=device)  # Change dtype to long
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32, device=device)  # Change dtype to long
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32, device=device)

    # Create PyTorch DataLoader for training and validation
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    return train_loader, val_loader

In [33]:
train_loader, val_loader = prepare_train_dataset(x_set, y_set, files)

using: cuda
validate with ['CIR_data_LOS_2_1.txt', 'CIR_data_NLOS_3_2.txt']


In [34]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        # 定义MLP的层
        self.w = 32
        self.emb = nn.Sequential(
            nn.Linear(1*64, self.w, bias=False),
            nn.ReLU(),
            nn.Linear(self.w, self.w//4, bias=False),
            nn.ReLU(),
            nn.Linear(self.w//4, 1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        # 定义前向传播
        x = self.emb(x)
        x = self.sigmoid(x)
        return x

In [35]:
from helper import count_parameters_bytes

# Create an instance of the PyTorch model and move it to GPU
model = PyTorchModel().to(device)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy loss
optimizer = optim.Adam(model.parameters(), lr=1e-5)

count_parameters_bytes(model)

Number of parameters: 9.25 KB


In [36]:
# Training loop
epochs = 300
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch [{epoch+1}/{epochs}] Train Loss: {total_loss/len(train_loader)}")

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        for batch_x_val, batch_y_val in val_loader:
            val_outputs = model(batch_x_val)
            val_loss += criterion(val_outputs, batch_y_val.view(-1, 1)).item()
            val_predictions = (val_outputs > 0.5).float()
            correct_predictions += (val_predictions == batch_y_val.view(-1, 1)).sum().item()
            total_samples += batch_y_val.size(0)
        val_accuracy = correct_predictions / total_samples
        print(f"Epoch [{epoch+1}/{epochs}] Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy * 100:.2f}%")

Epoch [1/100] Train Loss: 0.6918036407274162
Epoch [1/100] Validation Loss: 0.6909257295940604, Validation Accuracy: 56.22%
Epoch [2/100] Train Loss: 0.6911586757566108
Epoch [2/100] Validation Loss: 0.6907570644148758, Validation Accuracy: 55.65%
Epoch [3/100] Train Loss: 0.6904860498335658
Epoch [3/100] Validation Loss: 0.6905539818108082, Validation Accuracy: 55.41%
Epoch [4/100] Train Loss: 0.6898457204848877
Epoch [4/100] Validation Loss: 0.6903108534004007, Validation Accuracy: 55.34%
Epoch [5/100] Train Loss: 0.689166292373693
Epoch [5/100] Validation Loss: 0.6900437261377063, Validation Accuracy: 54.97%
Epoch [6/100] Train Loss: 0.6884944019468383
Epoch [6/100] Validation Loss: 0.6897602145160947, Validation Accuracy: 54.57%
Epoch [7/100] Train Loss: 0.6878339085701757
Epoch [7/100] Validation Loss: 0.689462672919035, Validation Accuracy: 53.82%
Epoch [8/100] Train Loss: 0.6871365220122371
Epoch [8/100] Validation Loss: 0.6891586030168193, Validation Accuracy: 53.15%
Epoch [9/1

In [37]:
# 保存模型
torch.save(model.state_dict(), 'weights/cnn.pt')

In [38]:
w = [i.detach().cpu().numpy() for i in model.parameters()]
np.save("weights/np.npy", w)

d:\conda\envs\Pytorch\lib\site-packages\numpy\lib\npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
